# Model Code for: Water Budget Processing Module
####  Island Wide Nutrient Modeling and Quantification of Coastal Freshwater Discharge for Tutuila, American Samoa, Script 1 of 2

This script contains computational steps used to format existing output from a Tutuila Water Budget Model for use in the Tutuila-wide DIN loading model, which is documented in the report "Island Wide Nutrient Modeling and Quantification of Coastal Freshwater Discharge for Tutuila, American Samoa" as authored by Shuler Hydrologic LLC and delivered to the American Samoa Environmental Protection Agency. 

It should be noted that this script requires the Arcpy python module to run. This module requires that the user have a valid ESRI licence and is running this script in an environment where Arcpy is installed.  This script is optional to running the model, its output is contained in a single spreadsheet that is used by scrpt 2 of 2 as input. This spreadsheet has alreadt been generated using a cell size of 100 m and the 93 watershed boundaries used in the model. If the user wishes to modify calculation of water budget components for different boundaries, this script can be run and a different shapefile can be substituted for "AG_OSDS_PIGS_NATURAL_ACTIVE_WS.shp".

#### The Tutuila Water Budget Model is freely available here: 
https://github.com/UH-WRRC-SWB-model 

# Disclaimer
This script is provided as open-source software on the condition that neither Shuler Hydrologic LLC nor the American Samoa EPA shall be held liable for any damages resulting from the authorized or unauthorized use of the information. No warranty, expressed or implied, is made by Shuler Hydrologic LLC or the American Samoa EPA as to the accuracy and functioning of the program and related program material nor shall the fact of distribution constitute any such warranty and no responsibility is assumed by Shuler Hydrologic LLC in connection therewith.

This information is preliminary or provisional and is subject to revision.  This software is provided "AS IS."

Note that sensitive information, or datasets that are not publically available, are not posted in raw forms. The model code is licensed under the GNU General Public License v3.0 which is an open-access license designed to explicitly affirm any user’s unlimited permission to run, copy, and use the unmodified code from this repository. Please note that some raw datasets used in this work are not owned by the authors and may be subject to other licenses or conditions.

#### Prepared by:  
      Shuler Hydrologic LLC, Honolulu, HI 96826      
      
#### Prepared for:  
    American Samoa Environmental Protection Agency, P.O. Box PPA, Pago Pago, AS 96799

# Implementation

### 1) Import modules and set relative paths

In [1]:
import arcpy
import os
from arcpy.sa import *
arcpy.env.overwriteOutput = True # make sure overwrite files is on
import pandas as pd

SWB_asc_files = os.path.join('..', "Raw_data", 'GIS', 'SWB_asc_results' )

sheds = os.path.join("..", "Raw_data", "GIS", "AG_OSDS_PIGS_NATURAL_ACTIVE_WS.shp")

# make the screen bigger!
from IPython.display import display, HTML
display(HTML(data=""" <style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 85%; }
    div#maintoolbar-container { width: 99%; } </style> """))

### 2) Calculate water budget components summarized by watershed areas

In [2]:
Results = os.path.join('..', "Workspace", 'SWB2_results')
if not os.path.exists(Results):
    os.makedirs(Results)
    
Results_table = os.path.join(Results, "tables")
if not os.path.exists(Results_table):
    os.makedirs(Results_table)
    
# process each raster layer into a table of annual statistics  
""" note this is where need to modify to process into monthly statistics """
for i in (os.listdir(SWB_asc_files)):
    if i.endswith('.asc'):
        outZSaT = ZonalStatisticsAsTable(sheds, "Uniqe_ID", os.path.join(SWB_asc_files, i), os.path.join(Results_table, "temptab.dbf"))  # in arc format
        arcpy.TableToTable_conversion(outZSaT, Results_table, "Table_{}_1.csv".format(i)) 

# this block takes the parameters of interests, reads them and calculates water volumnes (m3/d) for each watershed

templist = []
intrest_list = ['Table_netinfiltration_annual.asc_1.csv', 'Table_runoff_annual.asc_1.csv', 'Table_rainfall_annual.asc_1.csv'] 
for c in intrest_list:
    data = pd.read_csv(os.path.join(Results_table, c))
    keyname = c.split("Table_")[1].split("_annual")[0]                   # parameter being worked on
    data[keyname] = (data['MEAN']*.0254/365) * data['AREA'] 
    temp_frame = data[["Uniqe_ID", keyname]]
    templist.append(temp_frame)
        
summarry_frame1 = data[['Uniqe_ID']]                                        # this is just sticking them all together into one dataframe
for i in templist:
    summarry_frame1 = summarry_frame1.merge(i, on ='Uniqe_ID', how='outer')
    
summarry_frame1 = summarry_frame1[summarry_frame1.netinfiltration != 0]   # some of the values turned out tot be 0, this just selects rows that have non-zero values

### 3) Calculate hydrologic pathways: Baseflow and SGD, in each basin from SWB model and partitioning coefficient

 applying the runoff to median flow to baseflow to total flow relationships derrived from the measured stream datasets (perualt, Wong, and UH ASPA)  we calculate hydrologic pathways
 
- SWB runoff is directly calculated
- SWB baseflow is 33 % +-17% of net infiltration
- SWB SGD is 67 +-48 % of net infil

In [3]:
summarry_frame1["SGD_m3pd"] = 0.67 * summarry_frame1["netinfiltration"]
summarry_frame1["SGD_std"] = 0.48 * summarry_frame1["netinfiltration"]
summarry_frame1["baseflow"] = 0.33 * summarry_frame1["netinfiltration"]
summarry_frame1["baseflow_std"] = 0.17 * summarry_frame1["netinfiltration"]

# write output file to be used as input by script 2 of 2
summarry_frame1.to_csv(os.path.join(Results, "WB_by_shed_m3pd_Fixed.csv"))